# Generate csv tweet data

In [ ]:
!pip install jsonlines

In [ ]:
import os
import jsonlines
import csv
import pandas as pd
import random


data = []

with open("sarcasm_detection_shared_task_twitter_training.jsonl","r+",encoding='utf-8') as f:
    for item in jsonlines.Reader(f):
        if item['label'] == 'SARCASM':
            item['label'] = 0
        else:
            item['label'] = 1

        data.append(item)

with open('twitter.csv', 'w+', newline='', encoding='utf-8') as f:
    csv_writer = csv.writer(f)
    csv_head = ['context', 'response', 'label']
    csv_writer.writerow(csv_head)

    for i in range(len(data)):
        data_row = [(''.join(data[i]['context'])).replace('@USER', '').replace('<URL>',''),
                    (data[i]['response']).replace('@USER', '').replace('<URL>',''), data[i]['label']]
        csv_writer.writerow(data_row)


total = pd.read_csv("twitter.csv")

index = set(range(total.shape[0]))
K_fold = []
for i in range(5):
    if i == 4:
        tmp = index
    else:
        tmp = random.sample(index, int(1.0 / 5 * total.shape[0]))
    index = index - set(tmp)
    print("Number:", len(tmp))
    K_fold.append(tmp)

for i in range(5):
    print("Fold", i)
    os.system("mkdir data_{}".format(i))
    dev_index = list(K_fold[i])
    train_index = []
    for j in range(5):
        if j != i:
            train_index += K_fold[j]
    total.iloc[train_index].to_csv("data_{}/train.csv".format(i))
    total.iloc[dev_index].to_csv("data_{}/dev.csv".format(i))

<ipython-input-2-4605ec277b2a>:38: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  tmp = random.sample(index, int(1.0 / 5 * total.shape[0]))


Number: 1000
Number: 1000
Number: 1000
Number: 1000
Number: 1000
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


# Classification using BERT

In [ ]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.8 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd

df = pd.read_csv("twitter.csv")
df.head(5)

,context,response,label
0,A minor child deserves privacy and should be k...,I don't get this .. obviously you do care o...,0
1,Why is he a loser ? He's just a Press Secret...,trying to protest about . Talking about him ...,0
2,Donald J . Trump is guilty as charged . The ev...,He makes an insane about of money from the ...,0
3,Jamie Raskin tanked Doug Collins . Collins loo...,Meanwhile Trump won't even release his SAT s...,0
4,Man ... y ’ all gone “ both sides ” the apocal...,Pretty Sure the Anti-Lincoln Crowd Claimed T...,0


In [ ]:
df['label'].value_counts()

0    2500
1    2500
Name: label, dtype: int64

In [ ]:
df['context_response'] = df['context'] + '' + df['response']
df.head(5)

,context,response,label,context_response
0,A minor child deserves privacy and should be k...,I don't get this .. obviously you do care o...,0,A minor child deserves privacy and should be k...
1,Why is he a loser ? He's just a Press Secret...,trying to protest about . Talking about him ...,0,Why is he a loser ? He's just a Press Secret...
2,Donald J . Trump is guilty as charged . The ev...,He makes an insane about of money from the ...,0,Donald J . Trump is guilty as charged . The ev...
3,Jamie Raskin tanked Doug Collins . Collins loo...,Meanwhile Trump won't even release his SAT s...,0,Jamie Raskin tanked Doug Collins . Collins loo...
4,Man ... y ’ all gone “ both sides ” the apocal...,Pretty Sure the Anti-Lincoln Crowd Claimed T...,0,Man ... y ’ all gone “ both sides ” the apocal...


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['context_response'],df['label'], stratify=df['label'])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4")

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers

# l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(512, activation='relu')(l)  # Adding a dense layer
# l = tf.keras.layers.Dropout(0.1)(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  {'input_word_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_type_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
118/118 [==============================] - 124s 902ms/step - loss: 0.7693 - accuracy: 0.5744 - precision: 0.5721 - recall: 0.5904
Epoch 2/10
118/118 [==============================] - 111s 938ms/step - loss: 0.6805 - accuracy: 0.5971 - precision: 0.5949 - recall: 0.6085
Epoch 3/10
118/118 [==============================] - 113s 956ms/step - loss: 0.6479 - accuracy: 0.6312 - precision: 0.6250 - recall: 0.6560
Epoch 4/10
115/118 [============================>.] - ETA: 2s - loss: 0.6398 - accuracy: 0.6361 - precision: 0.6332 - recall: 0.6425

In [ ]:
model.evaluate(X_test, y_test)

40/40 [==============================] - 13s 317ms/step - loss: 0.5308 - accuracy: 0.7352 - precision: 0.8012 - recall: 0.6256


[0.5307614803314209,
 0.7351999878883362,
 0.8012295365333557,
 0.6255999803543091]